In [1]:
from Init3 import *

Imported all modules, QCoDeS version: 0.52.0 initialized
找到 2 個 .db 文件:
1. c:\Users\admin\Documents\GitHub\QCoDeS_local\personal_scripts\Albert\PtTe2_flux015B\003\LT\LT_2025-05-18_01.db
2. c:\Users\admin\Documents\GitHub\QCoDeS_local\personal_scripts\Albert\PtTe2_flux015B\003\RT\RT_2025-04-02_01.db
初始化數據庫: c:\Users\admin\Documents\GitHub\QCoDeS_local\personal_scripts\Albert\PtTe2_flux015B\003\LT\LT_2025-05-18_01.db


In [49]:
def analyze_iv_curve(df, x_param, y_param, dvdi_param='dV_dI', i_bar=110e-6):
    """
    Analyze the IV curve to detect critical currents and calculate resistances.
    
    Args:
        df (pd.DataFrame): DataFrame containing the IV data.
        x_param (str): Name of the current parameter.
        y_param (str): Name of the voltage parameter.
        dvdi_param (str, optional): Name of the dV/dI parameter. Defaults to 'dV_dI'.
    
    Returns:
        dict: Analysis results including Ic, Ir, resistance fits, and peak information.
    """
    # Compute dV/dI if not present
    if dvdi_param not in df.columns:
        df[dvdi_param] = np.gradient(df[y_param], df[x_param])
    # Filter out df[x_param] < filter bar values
    df = df[df[x_param] < i_bar]
    
    # Sort the DataFrame by x_param for consistent analysis
    df = df.sort_values(by=x_param).reset_index(drop=True)
    
    # Step 1: Determine scan type
    is_unidirectional = determine_scan_type(df, x_param)
    
    # Step 2: Find peaks in dV/dI
    positive_peak_indices, negative_peak_indices, current_0_index = find_peaks_in_dvdi(df, x_param, dvdi_param)
    
    # Step 3: Identify critical currents
    Ic, Ir = identify_critical_currents(df, x_param, dvdi_param, positive_peak_indices, negative_peak_indices, is_unidirectional)
    
    # Step 4: Define data regions
    df_0, df_1, df_between = get_data_regions(df, x_param, Ic, Ir, is_unidirectional)
    
    # Step 5: Perform linear fits
    R_fit0, intercept_0 = perform_linear_fit(df_0, x_param, y_param)
    R_fit1, intercept_1 = perform_linear_fit(df_1, x_param, y_param)
    R_fit_SC, intercept_SC = perform_linear_fit(df_between, x_param, y_param)
    
    # Step 6: Compute additional metrics
    R_fit = (R_fit0 + R_fit1) / 2
    IcRn = Ic * R_fit
    
    # Return comprehensive results
    return {
        'Ic': Ic,
        'Ir': Ir,
        'current_0_index': current_0_index,
        'R_fit': R_fit,
        'R_fit0': R_fit0,
        'R_fit1': R_fit1,
        'R_fit_SC': R_fit_SC,
        'IcRn': IcRn,
        'is_unidirectional': is_unidirectional,
        'fits': {
            'fit_0': (R_fit0, intercept_0),
            'fit_1': (R_fit1, intercept_1),
            'fit_between': (R_fit_SC, intercept_SC)
        },
        'peaks': {
            'positive_peak_currents': df[x_param].loc[positive_peak_indices].values,
            'positive_peak_R': df[dvdi_param].loc[positive_peak_indices].values,
            'negative_peak_currents': df[x_param].loc[negative_peak_indices].values,
            'negative_peak_R': df[dvdi_param].loc[negative_peak_indices].values
        }
    }

In [ ]:
dataid = 307
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
clear_output(wait=True)
fig1.show()
fig2.show()
fig3.show()

In [ ]:
dataid = 197 # East(E) 0°
dataid = 196 # NorthEast(NE) 45°
dataid = 178 # North(N) 90°
dataid = 180 # NorthWest(NW) 135°
dataid = 186 # West(W) 180°
dataid = 185 # SouthWest(SW) 225°
dataid = 188 # South(S) 270°
dataid = 191 # SouthEast(SE) 315°

In [64]:
# Critical current Ic comparison with fixed In-plane field value but different angles
f=0
ByRange = 5e-3
IcMax_df = pd.DataFrame(columns=["angle", "IcMax","By", "dataid"])
dataid = 196 # NorthEast(NE) 45°
angle = 45
Ic_offset = angle*1e-6*f
ByOffset = 2e-3
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
# Find the param_value where Ic is maximal
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"]
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig = go.Figure()
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°NE{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
dataid = 197 # East(E) 0°
angle = 0
Ic_offset = angle*1e-6*f
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"] 
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°E{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
dataid = 191 # SouthEast(SE) 315°
angle = 315
Ic_offset = angle*1e-6*f
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"] 
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°SE{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
dataid = 188 # South(S) 270°
angle = 270
Ic_offset = angle*1e-6*f
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"] 
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°S{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
dataid = 185 # SouthWest(SW) 225°
angle = 225
Ic_offset = angle*1e-6*f
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"] 
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°SW{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
dataid = 186 # West(W) 180°
angle = 180
Ic_offset = angle*1e-6*f
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"] 
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°W{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
dataid = 180 # NorthWest(NW) 135°
angle = 135
Ic_offset = angle*1e-6*f
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"] 
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°NW{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
dataid = 178 # North(N) 90°
angle = 90
Ic_offset = angle*1e-6*f
fig1, fig2, fig3 , Ic_df= plot_heatmaps(dataid)
# add Max Ic of Ic_df to IcMax_df label with dataid and angle
# Filter out the Ic_df where Ic_df["param_value"] is between 0-ByRange and 0+ByRange
Ic_df = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)]
By = Ic_df.loc[Ic_df["Ic"].idxmax(), "param_value"] 
new_row = pd.DataFrame([{"angle": angle, "IcMax": Ic_df["Ic"].max(),"By": By, "dataid": dataid}])
IcMax_df = pd.concat([IcMax_df, new_row], ignore_index=True)
clear_output(wait=True)
fig.add_trace(go.Scatter(
        x=Ic_df["param_value"], # Out-off-plane field y_field values, unit: T
        y=Ic_df["Ic"]+Ic_offset, # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°N{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
# add IcMax_df to fig
fig.add_trace(go.Scatter(
        x=IcMax_df["By"], # angle values, unit: degree
        y=IcMax_df["IcMax"]+IcMax_df["angle"]*1e-6*f, # Critical current Ic values, unit: A
        mode='lines+markers',
        name='IcMax',
        marker=dict(size=5),
        line=dict(width=2)
    ))
fig.show()

In [39]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByOffset, ByRange, f_factor):
    """
    Processes data for a given angle and dataid, updates the IcMax_df,
    and adds a trace to the figure.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values.
        ByRange (float): The range to filter 'param_value'.
        f_factor (float): The factor 'f' used in Ic_offset calculation.

    Returns:
        tuple: Updated (fig, IcMax_df)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor
    
    # Simulate plot_heatmaps if it's not available in this environment
    # In your actual code, you would call your plot_heatmaps function:
    # fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid)
    
    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    # Replace this with your actual plot_heatmaps call and Ic_df structure
    # For demonstration, creating a dummy Ic_df
    if 'plot_heatmaps' not in globals():
        print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 10
        _param_values = [(i - _data_points/2) * ByRange / (_data_points/2) for i in range(_data_points)] # Symetric around 0
        _ic_values = [abs(5e-6 - abs(pv)*1e-3 + (angle/90)*1e-7) for pv in _param_values] # Dummy Ic values
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        # This is where your actual data loading happens
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid) 
    # --- End of Placeholder ---

    # Filter out the Ic_df
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy() # Use .copy() to avoid SettingWithCopyWarning

    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange {ByRange}. Skipping this data point for IcMax and plot trace.")
        return fig, IcMax_df

    # Find the param_value where Ic is maximal
    # Ensure 'Ic' column exists and is numeric
    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        return fig, IcMax_df
        
    By = Ic_df_filtered.loc[Ic_df_filtered["Ic"].idxmax(), "param_value"]
    Ic_max_value = Ic_df_filtered["Ic"].max()
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    # Add trace to the main figure
    fig.add_trace(go.Scatter(
        x=Ic_df_filtered["param_value"]+ByOffset,  # Out-off-plane field y_field values, unit: T
        y=Ic_df_filtered["Ic"] + Ic_offset,  # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°{angle_label}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    clear_output(wait=True) # Clears the output of the cell, useful in notebooks
    return fig, IcMax_df

# --- Main Script ---
f = 2
ByOffset = 2e-3
ByRange = 5e-3
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

# Define data points as a list of dictionaries for easier iteration

data_points = [
    {"dataid": 192, "angle": 0, "label": "E"},
    {"dataid": 194, "angle": 45, "label": "NE"},
    {"dataid": 177, "angle": 90, "label": "N"},
    {"dataid": 179, "angle": 135, "label": "NW"},
    {"dataid": 181, "angle": 180, "label": "W"},
    {"dataid": 183, "angle": 225, "label": "SW"},
    {"dataid": 187, "angle": 270, "label": "S"},
    {"dataid": 189, "angle": 315, "label": "SE"},
]

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByRange=ByRange,
        f_factor=f
    )

# Add IcMax_df trace to fig
if not IcMax_df.empty:
    fig.add_trace(go.Scatter(
        x=IcMax_df["By"],  # By values where Ic is maximal
        y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,  # Max Critical current Ic values, unit: A
        mode='lines+markers',
        name='IcMax',
        marker=dict(size=8, symbol='star'), # Made marker different for clarity
        line=dict(width=2, dash='dash') # Differentiated line style
    ))
else:
    print("IcMax_df is empty. Skipping the IcMax trend plot.")

# --- Figure Layout and Display ---
fig.update_layout(
    title="Ic-By@r60mT w/ In-Plane Field Angles",
    xaxis_title="Out-of-Plane Field (By) [T]",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (angle*{f}e-6)",
    legend_title="Angle",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=800,

)
fig.show()

print("\nFinal IcMax DataFrame:")
print(IcMax_df)


Final IcMax DataFrame:
  angle     IcMax      By dataid
0     0  0.000108 -0.0020    192
1    45  0.000102 -0.0020    194
2    90  0.000104 -0.0004    177
3   135  0.000110  0.0008    179
4   180  0.000114  0.0020    181
5   225  0.000100  0.0012    183
6   270  0.000108  0.0000    187
7   315  0.000116 -0.0020    189


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByOffset, ByRange, f_factor):
    """
    Processes data for a given angle and dataid, updates the IcMax_df,
    and adds a trace to the figure.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values.
        ByRange (float): The range to filter 'param_value'.
        f_factor (float): The factor 'f' used in Ic_offset calculation.

    Returns:
        tuple: Updated (fig, IcMax_df)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor
    
    # Simulate plot_heatmaps if it's not available in this environment
    # In your actual code, you would call your plot_heatmaps function:
    # fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid)
    
    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    # Replace this with your actual plot_heatmaps call and Ic_df structure
    # For demonstration, creating a dummy Ic_df
    if 'plot_heatmaps' not in globals():
        print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 10
        _param_values = [(i - _data_points/2) * ByRange / (_data_points/2) for i in range(_data_points)] # Symetric around 0
        _ic_values = [abs(5e-6 - abs(pv)*1e-3 + (angle/90)*1e-7) for pv in _param_values] # Dummy Ic values
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        # This is where your actual data loading happens
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid) 
    # --- End of Placeholder ---

    # Filter out the Ic_df
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy() # Use .copy() to avoid SettingWithCopyWarning

    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange {ByRange}. Skipping this data point for IcMax and plot trace.")
        return fig, IcMax_df

    # Find the param_value where Ic is maximal
    # Ensure 'Ic' column exists and is numeric
    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        return fig, IcMax_df
        
    By = Ic_df_filtered.loc[Ic_df_filtered["Ic"].idxmax(), "param_value"]
    Ic_max_value = Ic_df_filtered["Ic"].max()
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    # Add trace to the main figure
    fig.add_trace(go.Scatter(
        x=Ic_df_filtered["param_value"]+ByOffset,  # Out-off-plane field y_field values, unit: T
        y=Ic_df_filtered["Ic"] + Ic_offset,  # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°{angle_label}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    clear_output(wait=True) # Clears the output of the cell, useful in notebooks
    return fig, IcMax_df

# --- Main Script ---
f = 0.2
ByRange = 10e-3
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

# Define data points as a list of dictionaries for easier iteration
data_points = [
    {"dataid": 144, "angle": 0, "label": "O", "ByOffset": 0e-3},
    {"dataid": 197, "angle": 0, "label": "E", "ByOffset": 2e-3+5.5e-6},
    {"dataid": 196, "angle": 45, "label": "NE", "ByOffset": 2e-3+500e-9},
    {"dataid": 178, "angle": 90, "label": "N", "ByOffset": 0e-3+6e-6},
    {"dataid": 180, "angle": 135, "label": "NW", "ByOffset": 0e-3+7.5e-6},
    {"dataid": 186, "angle": 180, "label": "W", "ByOffset": -2e-3},
    {"dataid": 185, "angle": 225, "label": "SW", "ByOffset": -1.2e-3+6e-6},
    {"dataid": 188, "angle": 270, "label": "S", "ByOffset": 0e-3+3.5e-6},
    {"dataid": 191, "angle": 315, "label": "SE", "ByOffset": 2e-3+7.5e-6},
]

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByOffset=point["ByOffset"],
        ByRange=ByRange,
        f_factor=f
    )

# Add IcMax_df trace to fig
if not IcMax_df.empty:
    fig.add_trace(go.Scatter(
        x=IcMax_df["By"],  # By values where Ic is maximal
        y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,  # Max Critical current Ic values, unit: A
        mode='lines+markers',
        name='IcMax',
        marker=dict(size=8, symbol='star'), # Made marker different for clarity
        line=dict(width=2, dash='dash') # Differentiated line style
    ))
else:
    print("IcMax_df is empty. Skipping the IcMax trend plot.")

# --- Figure Layout and Display ---
fig.update_layout(
    title="<b>|005-1|Ic-By@r60mT| w/ In-Plane Field Angles",
    xaxis_title="Out-of-Plane Field (By) [T]",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (angle*{f}e-6)",
    legend_title="Angle",
    hovermode="x unified",
    template="plotly_white",
    width=1600,
    height=800,

)
fig.show()

print("\nFinal IcMax DataFrame:")
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByOffset, ByRange, f_factor):
    """
    Processes data for a given angle and dataid, updates the IcMax_df,
    and adds a trace to the figure.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values.
        ByRange (float): The range to filter 'param_value'.
        f_factor (float): The factor 'f' used in Ic_offset calculation.

    Returns:
        tuple: Updated (fig, IcMax_df)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor
    
    # Simulate plot_heatmaps if it's not available in this environment
    # In your actual code, you would call your plot_heatmaps function:
    # fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid)
    
    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    # Replace this with your actual plot_heatmaps call and Ic_df structure
    # For demonstration, creating a dummy Ic_df
    if 'plot_heatmaps' not in globals():
        print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 10
        _param_values = [(i - _data_points/2) * ByRange / (_data_points/2) for i in range(_data_points)] # Symetric around 0
        _ic_values = [abs(5e-6 - abs(pv)*1e-3 + (angle/90)*1e-7) for pv in _param_values] # Dummy Ic values
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        # This is where your actual data loading happens
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid) 
    # --- End of Placeholder ---

    # Filter out the Ic_df
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy() # Use .copy() to avoid SettingWithCopyWarning

    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange {ByRange}. Skipping this data point for IcMax and plot trace.")
        return fig, IcMax_df

    # Find the param_value where Ic is maximal
    # Ensure 'Ic' column exists and is numeric
    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        return fig, IcMax_df
        
    By = Ic_df_filtered.loc[Ic_df_filtered["Ic"].idxmax(), "param_value"]
    Ic_max_value = Ic_df_filtered["Ic"].max()
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    # Add trace to the main figure
    fig.add_trace(go.Scatter(
        x=Ic_df_filtered["param_value"]+ByOffset,  # Out-off-plane field y_field values, unit: T
        y=Ic_df_filtered["Ic"] + Ic_offset,  # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°{angle_label}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    clear_output(wait=True) # Clears the output of the cell, useful in notebooks
    return fig, IcMax_df

# --- Main Script ---
f = 0.2
ByRange = 10e-3
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

# Define data points as a list of dictionaries for easier iteration
data_points = [
    {"dataid": 144, "angle": 0, "label": "O", "ByOffset": 0e-3},
    {"dataid": 197, "angle": 0, "label": "E", "ByOffset": 2e-3+5.5e-6},
    {"dataid": 196, "angle": 45, "label": "NE", "ByOffset": 2e-3+500e-9},
    {"dataid": 178, "angle": 90, "label": "N", "ByOffset": 0e-3+6e-6},
    {"dataid": 180, "angle": 135, "label": "NW", "ByOffset": 0e-3+7.5e-6},
    {"dataid": 186, "angle": 180, "label": "W", "ByOffset": -2e-3},
    {"dataid": 185, "angle": 225, "label": "SW", "ByOffset": -1.2e-3+6e-6},
    {"dataid": 188, "angle": 270, "label": "S", "ByOffset": 0e-3+3.5e-6},
    {"dataid": 191, "angle": 315, "label": "SE", "ByOffset": 2e-3+7.5e-6},
]

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByOffset=point["ByOffset"],
        ByRange=ByRange,
        f_factor=f
    )

# Add IcMax_df trace to fig
if not IcMax_df.empty:
    fig.add_trace(go.Scatter(
        x=IcMax_df["By"],  # By values where Ic is maximal
        y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,  # Max Critical current Ic values, unit: A
        mode='lines+markers',
        name='IcMax',
        marker=dict(size=8, symbol='star'), # Made marker different for clarity
        line=dict(width=2, dash='dash') # Differentiated line style
    ))
else:
    print("IcMax_df is empty. Skipping the IcMax trend plot.")

# --- Figure Layout and Display ---
fig.update_layout(
    title="<b>|005-1|Ic-By@r60mT| w/ In-Plane Field Angles",
    xaxis_title="Out-of-Plane Field (By) [T]",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (angle*{f}e-6)",
    legend_title="Angle",
    hovermode="x unified",
    template="plotly_white",
    width=1600,
    height=800,

)
fig.show()

print("\nFinal IcMax DataFrame:")
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByOffset, ByRange, f_factor):
    """
    Processes data for a given angle and dataid, updates the IcMax_df,
    and adds a trace to the figure.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values.
        ByRange (float): The range to filter 'param_value'.
        f_factor (float): The factor 'f' used in Ic_offset calculation.

    Returns:
        tuple: Updated (fig, IcMax_df)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor
    
    # Simulate plot_heatmaps if it's not available in this environment
    # In your actual code, you would call your plot_heatmaps function:
    # fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid)
    
    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    # Replace this with your actual plot_heatmaps call and Ic_df structure
    # For demonstration, creating a dummy Ic_df
    if 'plot_heatmaps' not in globals():
        print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 10
        _param_values = [(i - _data_points/2) * ByRange / (_data_points/2) for i in range(_data_points)] # Symetric around 0
        _ic_values = [abs(5e-6 - abs(pv)*1e-3 + (angle/90)*1e-7) for pv in _param_values] # Dummy Ic values
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        # This is where your actual data loading happens
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid) 
    # --- End of Placeholder ---

    # Filter out the Ic_df
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy() # Use .copy() to avoid SettingWithCopyWarning

    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange {ByRange}. Skipping this data point for IcMax and plot trace.")
        return fig, IcMax_df

    # Find the param_value where Ic is maximal
    # Ensure 'Ic' column exists and is numeric
    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        return fig, IcMax_df
        
    By = Ic_df_filtered.loc[Ic_df_filtered["Ic"].idxmax(), "param_value"]
    Ic_max_value = Ic_df_filtered["Ic"].max()
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    # Add trace to the main figure
    fig.add_trace(go.Scatter(
        x=Ic_df_filtered["param_value"]+ByOffset,  # Out-off-plane field y_field values, unit: T
        y=Ic_df_filtered["Ic"] + Ic_offset,  # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°{angle_label}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    clear_output(wait=True) # Clears the output of the cell, useful in notebooks
    return fig, IcMax_df

# --- Main Script ---
f = 0.2
ByRange = 10e-3
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

# Define data points as a list of dictionaries for easier iteration
data_points = [
    {"dataid": 144, "angle": 0, "label": "O", "ByOffset": 0e-3},
    {"dataid": 197, "angle": 0, "label": "E", "ByOffset": 2e-3+5.5e-6},
    {"dataid": 196, "angle": 45, "label": "NE", "ByOffset": 2e-3+500e-9},
    {"dataid": 178, "angle": 90, "label": "N", "ByOffset": 0e-3+6e-6},
    {"dataid": 180, "angle": 135, "label": "NW", "ByOffset": 0e-3+7.5e-6},
    {"dataid": 186, "angle": 180, "label": "W", "ByOffset": -2e-3},
    {"dataid": 185, "angle": 225, "label": "SW", "ByOffset": -1.2e-3+6e-6},
    {"dataid": 188, "angle": 270, "label": "S", "ByOffset": 0e-3+3.5e-6},
    {"dataid": 191, "angle": 315, "label": "SE", "ByOffset": 2e-3+7.5e-6},
]

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByOffset=point["ByOffset"],
        ByRange=ByRange,
        f_factor=f
    )

# Add IcMax_df trace to fig
if not IcMax_df.empty:
    fig.add_trace(go.Scatter(
        x=IcMax_df["By"],  # By values where Ic is maximal
        y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,  # Max Critical current Ic values, unit: A
        mode='lines+markers',
        name='IcMax',
        marker=dict(size=8, symbol='star'), # Made marker different for clarity
        line=dict(width=2, dash='dash') # Differentiated line style
    ))
else:
    print("IcMax_df is empty. Skipping the IcMax trend plot.")

# --- Figure Layout and Display ---
fig.update_layout(
    title="<b>|005-1|Ic-By@r60mT| w/ In-Plane Field Angles",
    xaxis_title="Out-of-Plane Field (By) [T]",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (angle*{f}e-6)",
    legend_title="Angle",
    hovermode="x unified",
    template="plotly_white",
    width=1600,
    height=800,

)
fig.show()

print("\nFinal IcMax DataFrame:")
print(IcMax_df)


Final IcMax DataFrame:
  angle     IcMax        By dataid
0     0  0.000119 -0.000023    144
1     0  0.000117 -0.002028    197
2    45  0.000108 -0.002030    196
3    90  0.000115 -0.000029    178
4   135  0.000098 -0.000030    180
5   180  0.000117  0.001972    186
6   225  0.000103  0.001182    185
7   270  0.000113 -0.000029    188
8   315  0.000114 -0.002030    191


In [189]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByOffset, ByRange, f_factor):
    """
    Processes data for a given angle and dataid, updates the IcMax_df,
    and adds a trace to the figure.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values.
        ByRange (float): The range to filter 'param_value'.
        f_factor (float): The factor 'f' used in Ic_offset calculation.

    Returns:
        tuple: Updated (fig, IcMax_df)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor
    
    # Simulate plot_heatmaps if it's not available in this environment
    # In your actual code, you would call your plot_heatmaps function:
    # fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid)
    
    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    # Replace this with your actual plot_heatmaps call and Ic_df structure
    # For demonstration, creating a dummy Ic_df
    if 'plot_heatmaps' not in globals():
        print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 10
        _param_values = [(i - _data_points/2) * ByRange / (_data_points/2) for i in range(_data_points)] # Symetric around 0
        _ic_values = [abs(5e-6 - abs(pv)*1e-3 + (angle/90)*1e-7) for pv in _param_values] # Dummy Ic values
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        # This is where your actual data loading happens
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid) 
    # --- End of Placeholder ---

    # Filter out the Ic_df
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy() # Use .copy() to avoid SettingWithCopyWarning

    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange {ByRange}. Skipping this data point for IcMax and plot trace.")
        return fig, IcMax_df

    # Find the param_value where Ic is maximal
    # Ensure 'Ic' column exists and is numeric
    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        return fig, IcMax_df
        
    By = Ic_df_filtered.loc[Ic_df_filtered["Ic"].idxmax(), "param_value"]
    Ic_max_value = Ic_df_filtered["Ic"].max()
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    # Add trace to the main figure
    fig.add_trace(go.Scatter(
        x=Ic_df_filtered["param_value"]+ByOffset,  # Out-off-plane field y_field values, unit: T
        y=Ic_df_filtered["Ic"] + Ic_offset,  # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°{angle_label}{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    clear_output(wait=True) # Clears the output of the cell, useful in notebooks
    return fig, IcMax_df

# --- Main Script ---
f = 0.2
ByRange = 10e-3
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

# Define data points as a list of dictionaries for easier iteration
data_points = [
    {"dataid": 251, "angle": 0, "label": "O", "ByOffset": 0e-3+12e-6},
    {"dataid": 230, "angle": 0, "label": "E", "ByOffset": 2.5e-3+13.7e-6},
    {"dataid": 235, "angle": 45, "label": "NE", "ByOffset": 2e-3+10e-6+700e-9},
    {"dataid": 240, "angle": 90, "label": "N", "ByOffset": 0.3e-3+9.1e-6},
    {"dataid": 201, "angle": 90, "label": "N", "ByOffset": 0.e-3+22.5e-6},
    {"dataid": 242, "angle": 135, "label": "NW", "ByOffset": -1e-3+11.7e-6},
    {"dataid": 204, "angle": 135, "label": "NW", "ByOffset": -1e-3+66e-6},
    {"dataid": 254, "angle": 180, "label": "W", "ByOffset": -1.75e-3+10e-6},
    {"dataid": 206, "angle": 180, "label": "W", "ByOffset": -1.75e-3+274e-6},
    {"dataid": 252, "angle": 225, "label": "SW", "ByOffset": -1.25e-3+8.2e-6},
    {"dataid": 213, "angle": 270, "label": "S", "ByOffset": 0.1e-3+9.7e-6},
    {"dataid": 208, "angle": 225, "label": "SW", "ByOffset": -1.25e-3+261.5e-6},
    {"dataid": 223, "angle": 315, "label": "SE", "ByOffset": 1.75e-3+9.5e-6},
]

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByOffset=point["ByOffset"],
        ByRange=ByRange,
        f_factor=f
    )

# Add IcMax_df trace to fig
# if not IcMax_df.empty:
    # fig.add_trace(go.Scatter(
    #     x=IcMax_df["By"],  # By values where Ic is maximal
    #     y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,  # Max Critical current Ic values, unit: A
    #     mode='lines+markers',
    #     name='IcMax',
    #     marker=dict(size=8, symbol='star'), # Made marker different for clarity
    #     line=dict(width=2, dash='dash') # Differentiated line style
    # ))
# else:
print("IcMax_df is empty. Skipping the IcMax trend plot.")

# --- Figure Layout and Display ---
fig.update_layout(
    title="<b>|005-2|Ic-By@r60mT| w/ In-Plane Field Angles",
    xaxis_title="Out-of-Plane Field (By) [T]",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (angle*{f}e-6)",
    legend_title="Angle",
    hovermode="x unified",
    template="plotly_white",
    # width=1600,
    # height=1000,

)
fig.show()

print("\nFinal IcMax DataFrame:")
print(IcMax_df)

IcMax_df is empty. Skipping the IcMax trend plot.



Final IcMax DataFrame:
   angle     IcMax        By dataid
0      0  0.000049  0.000013    251
1      0  0.000041 -0.002488    230
2     45  0.000042 -0.001985    235
3     90  0.000048 -0.000315    240
4     90  0.000056 -0.000030    201
5    135  0.000044  0.001007    242
6    135  0.000048  0.000971    204
7    180  0.000052  0.001735    254
8    180  0.000050  0.001471    206
9    225  0.000056  0.001235    252
10   270  0.000045 -0.000103    213
11   225  0.000049  0.000976    208
12   315  0.000044 -0.001765    223


In [45]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition (Modified) ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByRange, f_factor):
    """
    Processes data for a given angle and dataid, updates the IcMax_df,
    adds a trace to the figure, and returns the filtered Ic_df.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values.
        ByRange (float): The range to filter 'param_value'.
        f_factor (float): The factor 'f' used in Ic_offset calculation.

    Returns:
        tuple: Updated (fig, IcMax_df, Ic_df_filtered)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor
    
    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    if 'plot_heatmaps' not in globals():
        # print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 10 # Number of points for By
        # Symmetrically distributed param_values around 0, within ByRange
        _param_values = [(val / (_data_points / 2 -1)) * ByRange if _data_points > 1 else 0 for val in range(-int(_data_points/2)+1, int(_data_points/2)+1)]
        _param_values = sorted(list(set(_param_values))) # Ensure unique and sorted values
        if not _param_values and ByRange == 0: # Handle edge case where ByRange is 0
             _param_values = [0.0]
        elif not _param_values and _data_points ==1:
             _param_values = [0.0]


        # Dummy Ic values that somewhat decrease away from param_value = 0 and vary with angle
        _ic_values = [
            max(0, 5e-6 - abs(pv) * 5e-4 - (abs(angle - 180)/180)*2e-6 + (0 if pv !=0 else 0.5e-6 * ( (180-abs(angle-180))/180 ) ) ) 
            for pv in _param_values
        ]
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
        # fig1, fig2, fig3 are not used by this function if we only use dummy data
    else:
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid)
    # --- End of Placeholder ---

    Ic_df_filtered = Ic_df[(Ic_df["param_value"] <= ByRange) & (Ic_df["param_value"] >= -ByRange)].copy()

    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange {ByRange}. Skipping this data point for IcMax and plot trace.")
        return fig, IcMax_df, Ic_df_filtered # Return empty DF for consistency

    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        # Return the current state and an empty df for Ic_df_filtered to signal error
        return fig, IcMax_df, pd.DataFrame(columns=['param_value', 'Ic']) 

    By = Ic_df_filtered.loc[Ic_df_filtered["Ic"].idxmax(), "param_value"]
    Ic_max_value = Ic_df_filtered["Ic"].max()
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    fig.add_trace(go.Scatter(
        x=Ic_df_filtered["param_value"],
        y=Ic_df_filtered["Ic"] + Ic_offset,
        mode='lines+markers',
        name=f'{angle}°{angle_label}{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    # clear_output(wait=True) # Commented out for potentially faster processing in loops if output is large
    return fig, IcMax_df, Ic_df_filtered # Return the filtered DataFrame

# --- Main Script ---
f = 2
ByRange = 5e-3 # Max By value for X-axis of heatmap and for filtering individual traces
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

data_points = [
    {"dataid": 194, "angle": 45, "label": "NE"},
    {"dataid": 192, "angle": 0, "label": "E"},
    {"dataid": 189, "angle": 315, "label": "SE"},
    {"dataid": 187, "angle": 270, "label": "S"},
    {"dataid": 183, "angle": 225, "label": "SW"},
    {"dataid": 181, "angle": 180, "label": "W"},
    {"dataid": 179, "angle": 135, "label": "NW"},
    {"dataid": 177, "angle": 90, "label": "N"},
]

all_dfs_for_heatmap = [] # To collect DataFrames for the heatmap

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df, current_Ic_df_filtered = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByRange=ByRange,
        f_factor=f
    )
    if current_Ic_df_filtered is not None and not current_Ic_df_filtered.empty:
        # Select only necessary columns and add the angle for heatmap construction
        df_slice = current_Ic_df_filtered[['param_value', 'Ic']].copy()
        df_slice['angle'] = point["angle"]
        all_dfs_for_heatmap.append(df_slice)

clear_output(wait=True) # Clear output once after all processing

# Add IcMax_df trace to the scatter fig
if not IcMax_df.empty:
    fig.add_trace(go.Scatter(
        x=IcMax_df["By"],
        y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,
        mode='lines+markers',
        name='IcMax Trend',
        marker=dict(size=8, symbol='star'),
        line=dict(width=2, dash='dash')
    ))
else:
    print("IcMax_df is empty. Skipping the IcMax trend plot.")

fig.update_layout(
    title="Critical Current (Ic) vs. Out-of-Plane Field (By) at Different Angles",
    xaxis_title=f"Out-of-Plane Field (By) [T] (within ±{ByRange:.0e} T)",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (Offset = angle * 1e-6 * {f})",
    legend_title="Measurement Angle & ID",
    hovermode="x unified",
    width=800,
    height=800,
    template="plotly_white"
)
fig.show()

print("\nFinal IcMax DataFrame:")
print(IcMax_df)

# --- Heatmap Generation ---
if all_dfs_for_heatmap:
    heatmap_source_data = pd.concat(all_dfs_for_heatmap, ignore_index=True)
    
    if not heatmap_source_data.empty and 'param_value' in heatmap_source_data and 'angle' in heatmap_source_data and 'Ic' in heatmap_source_data:
        try:
            # Create the pivot table: angles as rows, By (param_value) as columns, Ic as values
            pivot_table = heatmap_source_data.pivot_table(
                index='angle', 
                columns='param_value', 
                values='Ic'
            )
            
            # Sort index (angles) and columns (By values) for an ordered heatmap
            pivot_table = pivot_table.sort_index(axis=0)  # Sort by angle (Y-axis)
            pivot_table = pivot_table.sort_index(axis=1)  # Sort by param_value (X-axis)

            heatmap_fig = go.Figure(data=go.Heatmap(
                z=pivot_table.values,    # 2D array of Ic values
                x=pivot_table.columns,   # By (param_value)
                y=pivot_table.index,     # Angles
                colorscale='RdBu',    # Example colorscale
                colorbar=dict(title='Ic [A]'),
                connectgaps=False,
            ))
            
            heatmap_fig.update_layout(
                title='Critical Current (Ic) Heatmap@r60mT',
                xaxis_title=f"Out-of-Plane Field (By) [T] (within ±{ByRange:.0e} T)",
                yaxis_title='Angle [degrees]',
                # xaxis_type='linear' # Default, should work if param_value is numeric
                yaxis_type='category', # Angles are distinct categories
                width=800,
                height=800,
                template="plotly_white"
            )
            heatmap_fig.show()
            
        except Exception as e:
            print(f"Could not generate heatmap. Error: {e}")
            print("Sample of data intended for pivot table:")
            print(heatmap_source_data.head())
    else:
        print("Source data for heatmap is empty or missing required columns ('param_value', 'angle', 'Ic').")
else:
    print("No data was collected to generate the heatmap (all_dfs_for_heatmap list is empty).")


Final IcMax DataFrame:
  angle     IcMax      By dataid
0    45  0.000102 -0.0020    194
1     0  0.000108 -0.0020    192
2   315  0.000116 -0.0020    189
3   270  0.000108  0.0000    187
4   225  0.000100  0.0012    183
5   180  0.000114  0.0020    181
6   135  0.000110  0.0008    179
7    90  0.000104 -0.0004    177


In [32]:
IcMax_df

,angle,IcMax,By,dataid
0,45,0.000102,-0.0020,194
1,0,0.000108,-0.0020,192
2,315,0.000116,-0.0020,189
3,270,0.000108,0.0000,187
4,225,0.000100,0.0012,183
5,180,0.000114,0.0020,181
6,135,0.000110,0.0008,179
7,90,0.000104,-0.0004,177


In [81]:
# range=[-0.05e-3, 0.05e-3]
fig1.data[0].update(zmin=-10e-6, zmax=0e-6)
# fig1.update_xaxes(range=range) 
fig1.show()
fig2.data[0].update(zmin=0, zmax=0.3)
# fig2.update_xaxes(range=range)
fig2.show()
# fig3.update_xaxes(range=range) 
fig3.update_yaxes(range=[130e-6, 140e-6])
fig3.show()

In [96]:
# Critical current IcMax comparison with fixed In-plane field value but different angles

# Re-importing your existing functions to ensure they are defined in this block
# (assuming you are running this in an environment where the previous cells are executed)
# from __main__ import info_df, analyze_iv_curve, get_dataset_info, SI, ureg, FitResult, polyfit, constfit, print_fit_result, _display_time, cached_load_dataset

def get_ic_dataframe(runids):
    """
    Extracts critical current (Ic) and other relevant parameters for a list of QCoDeS run IDs
    and returns them as a Pandas DataFrame. It mimics the Ic extraction logic
    used for the fig3 plot in plot_heatmaps to get a single representative Ic value per runid.

    Args:
        runids (list of int): A list of QCoDeS run IDs to analyze.

    Returns:
        pd.DataFrame: A DataFrame containing 'runid', 'Ic', 'Ir', 'R_fit', and other
                      analysis results for each run. Returns an empty DataFrame if no data is found.
    """
    if not isinstance(runids, list):
        print("Warning: 'runids' should be a list. Converting to list.")
        runids = [runids]

    all_ic_data = []

    print(f"Starting Ic extraction for {len(runids)} runs...")
    for runid in tqdm(runids, desc="Processing runs"):
        try:
            # Load dataset information and DataFrame
            setpoint_params, dependent_param, param_info, df = info_df(runid)
            exp_name, sample_name, param_units, start_time, completed_time, run_time, display_time = get_dataset_info(runid)

            # Identify current and voltage parameters from the dataset
            current_param = next((p for p in setpoint_params if 'curr' in p.lower()), None)
            voltage_param = dependent_param # By convention, dependent_param is usually voltage

            if current_param is None:
                print(f"Warning: No current parameter found in runid {runid}. Skipping.")
                continue

            # Find the 'outer loop' parameter which is typically the first setpoint parameter
            # that is NOT the current_param. This is what you were using as x_param in plot_heatmaps.
            outer_loop_param = None
            for sp in setpoint_params:
                if sp != current_param:
                    outer_loop_param = sp
                    break

            ic_values_for_run = []
            ir_values_for_run = []

            # Determine the overall scan direction of the current for the run
            # This logic comes from your plot_heatmaps to decide 'Ic+' or 'Ic-'
            y_min_overall, y_max_overall = df[current_param].min(), df[current_param].max()
            if abs(y_max_overall) > abs(y_min_overall) and y_max_overall > 0:
                scan_direction_for_run = 'Ic+' # Positive dominant sweep
            elif abs(y_min_overall) > abs(y_max_overall) and y_min_overall < 0:
                scan_direction_for_run = 'Ic-' # Negative dominant sweep
            else:
                # If symmetric or mostly zero, default to positive or handle as needed
                scan_direction_for_run = 'Ic+'
                
            # If there's an outer loop parameter, iterate over its unique values
            # and analyze each IV curve separately.
            if outer_loop_param:
                for outer_param_value, group_df in df.groupby(outer_loop_param):
                    # Ensure the group_df is sorted by current_param before analysis
                    group_df_sorted = group_df.sort_values(by=current_param).reset_index(drop=True)
                    analysis = analyze_iv_curve(group_df_sorted, x_param=current_param, y_param=voltage_param, dvdi_param='dV_dI')
                    
                    # Store both Ic and Ir from each analysis, as we might need both
                    ic_values_for_run.append(analysis['Ic'])
                    ir_values_for_run.append(analysis['Ir'])
            else:
                # If no outer loop param, it's a single IV curve for the whole run
                df_sorted = df.sort_values(by=current_param).reset_index(drop=True)
                analysis = analyze_iv_curve(df_sorted, x_param=current_param, y_param=voltage_param, dvdi_param='dV_dI')
                ic_values_for_run.append(analysis['Ic'])
                ir_values_for_run.append(analysis['Ir'])


            # Now, based on the overall scan direction, determine the representative Ic for the runid
            representative_ic = np.nan
            if scan_direction_for_run == 'Ic+':
                # Take the max positive Ic from all sweeps within this run
                valid_ics = [val for val in ic_values_for_run if np.isfinite(val) and val > 0]
                if valid_ics:
                    representative_ic = max(valid_ics)
                else:
                    # Fallback if no positive Ic found, perhaps use max absolute if it's the only one
                    valid_all_ics = [val for val in ic_values_for_run if np.isfinite(val)]
                    if valid_all_ics:
                         representative_ic = max(valid_all_ics, key=abs)

            elif scan_direction_for_run == 'Ic-':
                # Take the min (most negative) Ir from all sweeps within this run
                valid_irs = [val for val in ir_values_for_run if np.isfinite(val) and val < 0]
                if valid_irs:
                    representative_ic = min(valid_irs)
                else:
                    # Fallback if no negative Ir found
                    valid_all_irs = [val for val in ir_values_for_run if np.isfinite(val)]
                    if valid_all_irs:
                        representative_ic = min(valid_all_irs, key=abs) # If it's negative, `min(abs)` is largest magnitude negative
            
            # If no specific direction was determined or no valid Ic values:
            if np.isnan(representative_ic):
                # As a last resort, if all else fails, take the maximum absolute Ic from all
                # positive Ic values or negative Ir values that were found.
                all_valid_critical_currents = [val for val in ic_values_for_run + ir_values_for_run if np.isfinite(val)]
                if all_valid_critical_currents:
                    representative_ic = max(all_valid_critical_currents, key=abs)


            # Get some additional info to add to the DataFrame, e.g., sample_name
            # and potentially the value of the outer_loop_param if it's constant for the run
            outer_param_value_for_run = None
            if outer_loop_param and len(df[outer_loop_param].unique()) == 1:
                outer_param_value_for_run = df[outer_loop_param].iloc[0]


            run_data = {
                'runid': runid,
                'sample_name': sample_name,
                'exp_name': exp_name,
                'Ic': representative_ic,
                'scan_direction_inferred': scan_direction_for_run,
                f'outer_loop_param_{outer_loop_param}': outer_param_value_for_run if outer_loop_param else 'N/A'
                # You can add more aggregate data here if needed, e.g., avg R_fit for the run
            }
            all_ic_data.append(run_data)

        except Exception as e:
            print(f"Error processing runid {runid}: {e}")
            continue

    if not all_ic_data:
        print("No Ic data was extracted. Please check your runids and dataset structure.")
        return pd.DataFrame() # Return an empty DataFrame

    df_ic_results = pd.DataFrame(all_ic_data)
    
    # Reorder columns for better readability
    cols = ['runid', 'sample_name', 'exp_name', 'scan_direction_inferred', 'Ic']
    if outer_loop_param:
        cols.insert(4, f'outer_loop_param_{outer_loop_param}')
    
    # Ensure all columns exist before reordering
    cols = [c for c in cols if c in df_ic_results.columns]
    
    return df_ic_results[cols]

# Assuming you have runids where each runid might contain multiple IV sweeps
# (e.g., IV curves taken at different magnetic field values within one run).
# The 'Ic' in the output DataFrame will be the maximum Ic from that run.
my_measurement_runids = [151, 150] # Replace with your actual run IDs

ic_data_df = get_ic_dataframe(my_measurement_runids)

print("\n--- Extracted Ic Data DataFrame ---")
print(ic_data_df)

# You can now easily access specific columns, e.g.:
# print(ic_data_df['Ic'])
# print(ic_data_df[ic_data_df['sample_name'] == 'MySample1'])

Starting Ic extraction for 2 runs...


Processing runs:   0%|          | 0/2 [00:00<?, ?it/s]

Number of points for each parameter in dataset 151:
- meas_voltage_K1 (dependent): 51 points
- y_field          (setpoint): 51 unique points, from -1.00e-02 to 1.00e-02, step size: 4.00e-04
- appl_current     (setpoint): 181 unique points, from 0.00e+00 to 1.20e-04, step size: 6.67e-07
計算微分電阻 dV/dI，使用電壓: meas_voltage_K1 和電流: appl_current
已添加dV_dI列到數據框
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9231 entries, 0 to 9230
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   y_field          9231 non-null   float64
 1   appl_current     9231 non-null   float64
 2   meas_voltage_K1  9231 non-null   float64
 3   dV_dI            9231 non-null   float64
dtypes: float64(4)
memory usage: 288.6 KB
None
            y_field  appl_current  meas_voltage_K1        dV_dI
count  9.231000e+03   9231.000000      9231.000000  9231.000000
mean   3.848677e-19      0.000085         0.000026     0.358398
std    5.888160e-03      

In [97]:
ic_data_df

,runid,sample_name,exp_name,scan_direction_inferred,outer_loop_param_y_field,Ic
0,151,"005-1_28-27-30-29@r:32mT,θ:90.0°,φ:-18.4°@(30,...",I+VBy+,Ic+,None,0.000111
1,150,"005-1_28-27-30-29@r:30mT,θ:135.0°,φ:-0.1°@(21,...",I+VBy+,Ic+,None,0.000145


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByOffset, ByRange, f_factor):
    """
    Processes data for a given angle and dataid, updates the IcMax_df,
    and adds a trace to the figure.
    The 'param_value' for each trace is shifted so its minimum becomes 0,
    then ByOffset is added.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values.
        ByOffset (float): The offset to apply to the x-values AFTER aligning data start to 0.
        ByRange (float): The range to filter 'param_value'.
        f_factor (float): The factor 'f' used in Ic_offset calculation.

    Returns:
        tuple: Updated (fig, IcMax_df)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor
    
    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    if 'plot_heatmaps' not in globals():
        print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 10
        # Ensure _param_values have some variation to make min() meaningful
        _base_param_values = [(i - _data_points/2 + 0.5) * ByRange / (_data_points/2) for i in range(_data_points)] 
        # Add a small unique offset per dataid to simulate different raw start points
        _param_values = [pv + (dataid * 1e-4) for pv in _base_param_values] 
        _ic_values = [abs(5e-6 - abs(pv - (dataid*1e-4))*1e-3 + (angle/90)*1e-7) for pv in _param_values] # Dummy Ic values
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        # This is where your actual data loading happens
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid) 
    # --- End of Placeholder ---

    # Filter out the Ic_df
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < Ic_df["param_value"].min() + 2*ByRange) & (Ic_df["param_value"] > Ic_df["param_value"].min() - 2*ByRange)].copy()
    # More robust filtering based on a range around the mean or median if ByRange is relative
    # For now, let's assume ByRange is an absolute limit for the spread of relevant data
    # Or, if param_value is already somewhat centered, the original filter might be fine:
    # Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy()
    # Reinstating original filter logic if dummy data is designed for it:
    # For dummy data, ensure it's generated around some reference, then filter.
    # The dummy _param_values are generated centered around 0 (before the dataid shift).
    # If using real data, ensure ByRange is appropriate for raw param_values.
    # A simple way to ensure the filter is meaningful with shifted data:
    # Let's assume ByRange defines the *width* of data to consider.
    # For now, falling back to your original filter and adjusting dummy data if necessary.
    # The current dummy data is generated with a spread related to ByRange.
    # Re-evaluating the filter: The original filter `(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)`
    # assumes param_value is somewhat centered around 0, or that ByRange is a global limit.
    # Let's make the filter more adaptive or stick to the original if it has a specific meaning.
    # Given the alignment, let's filter first, then align.
    
    # Filter Ic_df based on param_value range *before* alignment
    # This assumes ByRange is a window relative to some central point of the raw data or absolute values
    # For simplicity, let's use the original filtering approach. If your raw data's `param_value`
    # is not centered around 0, this filter might need adjustment.
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy()


    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange {ByRange}. Check filter or data source. Skipping this data point.")
        return fig, IcMax_df

    # --- Alignment Step ---
    # Find the minimum of the 'param_value' in the filtered data
    min_param_value = Ic_df_filtered["param_value"].min()
    
    # Shift the 'param_value's so the minimum is 0, then add the ByOffset
    # This will be used for plotting the trace
    plot_x_values = (Ic_df_filtered["param_value"] - min_param_value) + ByOffset
    # --- End of Alignment Step ---

    # Ensure 'Ic' column exists and is numeric
    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        return fig, IcMax_df
        
    # Find the original param_value where Ic is maximal from the filtered (but not yet x-shifted for plot) data
    By_original = Ic_df_filtered.loc[Ic_df_filtered["Ic"].idxmax(), "param_value"]
    Ic_max_value = Ic_df_filtered["Ic"].max()
    
    # Apply the same alignment transformation to this By_original for storing in IcMax_df
    By_aligned_for_plot = (By_original - min_param_value) + ByOffset
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By_aligned_for_plot, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    # Add trace to the main figure using the aligned x_values
    fig.add_trace(go.Scatter(
        x=plot_x_values,  # Use the aligned and offsetted x-values
        y=Ic_df_filtered["Ic"] + Ic_offset,  # Critical current Ic values, unit: A
        mode='lines+markers',
        name=f'{angle}°{angle_label}{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    clear_output(wait=True) # Clears the output of the cell, useful in notebooks
    return fig, IcMax_df

# --- Main Script ---
f = 0.2
ByRange = 10e-3 # This range is used for filtering param_value *before* alignment.
                # Ensure it's appropriate for your raw data's scale.
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

# Define data points as a list of dictionaries for easier iteration
data_points = [
    {"dataid": 144, "angle": 0, "label": "O", "ByOffset": 0e-6},
    {"dataid": 197, "angle": 0, "label": "E", "ByOffset": 0e-6}, # Starts at x=0.0020055
    {"dataid": 196, "angle": 45, "label": "NE", "ByOffset": 0e-6},# Starts at x=0.0020005
    {"dataid": 178, "angle": 90, "label": "N", "ByOffset": 0e-6},  # Starts at x=0.000006
    {"dataid": 180, "angle": 135, "label": "NW", "ByOffset": 0e-6},# Starts at x=0.0000075
    {"dataid": 186, "angle": 180, "label": "W", "ByOffset": 0e-6},       # Starts at x=-0.002
    {"dataid": 185, "angle": 225, "label": "SW", "ByOffset": 0e-6},# Starts at x=-0.001194
    {"dataid": 188, "angle": 270, "label": "S", "ByOffset": 0e-6}, # Starts at x=0.0000035
    {"dataid": 191, "angle": 315, "label": "SE", "ByOffset": 0e-6},# Starts at x=0.0020075
]

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByOffset=point["ByOffset"], # This is now applied *after* data's own start is aligned to 0
        ByRange=ByRange,
        f_factor=f
    )

# Add IcMax_df trace to fig
# The "By" column in IcMax_df now contains the aligned and offsetted x-values
if not IcMax_df.empty:
    fig.add_trace(go.Scatter(
        x=IcMax_df["By"],  # These 'By' values are already transformed
        y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,
        mode='lines+markers',
        name='IcMax',
        marker=dict(size=8, symbol='star'),
        line=dict(width=2, dash='dash')
    ))
else:
    print("IcMax_df is empty. Skipping the IcMax trend plot.")

# --- Figure Layout and Display ---
# The x-axis title now refers to By values that have been aligned and then offset.
fig.update_layout(
    title="<b>|005-1|Ic-By@r60mT| w/ In-Plane Field Angles (Aligned Start + Offset)",
    xaxis_title=f"Out-of-Plane Field (By, aligned then offset) [T]",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (angle*{f}e-6)",
    legend_title="Angle",
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=800,
)
fig.show()

print("\nFinal IcMax DataFrame (By values are aligned then offset):")
print(IcMax_df)


Final IcMax DataFrame (By values are aligned then offset):
  angle     IcMax            By dataid
0     0  0.000119  7.500000e-06    144
1     0  0.000117  2.000000e-06    197
2    45  0.000108  5.000000e-07    196
3    90  0.000115  5.000000e-07    178
4   135  0.000098  0.000000e+00    180
5   180  0.000117  1.500000e-06    186
6   225  0.000103  1.150000e-05    185
7   270  0.000113  5.000000e-07    188
8   315  0.000114  5.000000e-07    191


In [262]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output

# Assume plot_heatmaps is defined elsewhere and works as expected
# from your_module import plot_heatmaps # Example of how you might import it

# --- Helper Function Definition ---
def process_and_plot_data(dataid, angle, angle_label, fig, IcMax_df, ByOffset, ByRange, f_factor):
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    Ic_offset = angle * 1e-6 * f_factor

    # --- Placeholder for plot_heatmaps and Ic_df generation ---
    if 'plot_heatmaps' not in globals():
        print(f"Warning: 'plot_heatmaps' function not found. Using dummy data for dataid {dataid}.")
        _data_points = 30
        _base_param_values = [(i - _data_points/2 + 0.5) * ByRange / (_data_points/2) for i in range(_data_points)]
        _param_values = [pv + (dataid * 1e-4) for pv in _base_param_values]
        # Add a synthetic peak for demonstration
        import numpy as np
        _ic_values = np.exp(-((np.array(_param_values)-0.002)**2)/(2*(0.001)**2)) * 5e-6 + 1e-6*np.random.rand(_data_points)
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        fig1, fig2, fig3, Ic_df = plot_heatmaps(dataid)
    # --- End of Placeholder ---

    # 1. 过滤掉前10个点
    Ic_df = Ic_df.iloc[10:].reset_index(drop=True)

    # 2. 只保留By在范围内的数据
    Ic_df_filtered = Ic_df[(Ic_df["param_value"] < ByRange) & (Ic_df["param_value"] > -ByRange)].copy()
    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering. Skipping.")
        return fig, IcMax_df

    # 3. 用find_peaks找第一个峰
    from scipy.signal import find_peaks
    peaks, _ = find_peaks(Ic_df_filtered["Ic"])
    if len(peaks) == 0:
        print(f"No peak found for dataid {dataid}. Using min(param_value) as alignment.")
        align_x = Ic_df_filtered["param_value"].min()
    else:
        first_peak_idx = peaks[0]
        align_x = Ic_df_filtered.iloc[first_peak_idx]["param_value"]

    # 4. 对齐By，使第一个peak在x=0，再加ByOffset
    plot_x_values = Ic_df_filtered["param_value"] - align_x + ByOffset

    # 5. 计算最大Ic及其By（同样对齐）
    idx_max = Ic_df_filtered["Ic"].idxmax()
    By_max = Ic_df_filtered.loc[idx_max, "param_value"]
    Ic_max_value = Ic_df_filtered.loc[idx_max, "Ic"]
    By_aligned_for_plot = By_max - align_x + ByOffset

    # 6. 更新IcMax_df
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By_aligned_for_plot, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)

    # 7. 画图
    fig.add_trace(go.Scatter(
        x=plot_x_values,
        y=Ic_df_filtered["Ic"] + Ic_offset,
        mode='lines+markers',
        name=f'{angle}°{angle_label}{dataid}',
        marker=dict(size=5),
        line=dict(width=2)
    ))

    clear_output(wait=True)
    return fig, IcMax_df

# --- Main Script ---
f = 0.2
ByRange = 10e-3 # This range is used for filtering param_value *before* alignment.
                # Ensure it's appropriate for your raw data's scale.
IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
fig = go.Figure()

# Define data points as a list of dictionaries for easier iteration
data_points = [
    {"dataid": 144, "angle": 0, "label": "O", "ByOffset": 0e-6},
    {"dataid": 197, "angle": 0, "label": "E", "ByOffset": 0e-6}, # Starts at x=0.0020055
    {"dataid": 196, "angle": 45, "label": "NE", "ByOffset": 3e-6},# Starts at x=0.0020005
    {"dataid": 178, "angle": 90, "label": "N", "ByOffset": 0e-6},  # Starts at x=0.000006
    {"dataid": 180, "angle": 135, "label": "NW", "ByOffset": 0e-6},# Starts at x=0.0000075
    {"dataid": 186, "angle": 180, "label": "W", "ByOffset": 0e-6},       # Starts at x=-0.002
    {"dataid": 185, "angle": 225, "label": "SW", "ByOffset": 0e-6},# Starts at x=-0.001194
    {"dataid": 188, "angle": 270, "label": "S", "ByOffset": 0e-6}, # Starts at x=0.0000035
    {"dataid": 191, "angle": 315, "label": "SE", "ByOffset": 0e-6},# Starts at x=0.0020075
]

# Loop through the data points and process them
for point in data_points:
    fig, IcMax_df = process_and_plot_data(
        dataid=point["dataid"],
        angle=point["angle"],
        angle_label=point["label"],
        fig=fig,
        IcMax_df=IcMax_df,
        ByOffset=point["ByOffset"], # This is now applied *after* data's own start is aligned to 0
        ByRange=ByRange,
        f_factor=f
    )

# Add IcMax_df trace to fig
# The "By" column in IcMax_df now contains the aligned and offsetted x-values
if not IcMax_df.empty:
    fig.add_trace(go.Scatter(
        x=IcMax_df["By"],  # These 'By' values are already transformed
        y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f,
        mode='lines+markers',
        name='IcMax',
        marker=dict(size=8, symbol='star'),
        line=dict(width=2, dash='dash')
    ))
else:
    print("IcMax_df is empty. Skipping the IcMax trend plot.")

# --- Figure Layout and Display ---
# The x-axis title now refers to By values that have been aligned and then offset.
fig.update_layout(
    title="<b>|005-1|Ic-By@r60mT| w/ In-Plane Field Angles (Aligned Start + Offset)",
    xaxis_title=f"Out-of-Plane Field (By, aligned then offset) [T]",
    yaxis_title=f"Critical Current (Ic) + Offset [A] (angle*{f}e-6)",
    legend_title="Angle",
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=800,
)
fig.show()

print("\nFinal IcMax DataFrame (By values are aligned then offset):")
print(IcMax_df)


Final IcMax DataFrame (By values are aligned then offset):
  angle     IcMax        By dataid
0     0  0.000119  0.000002    144
1     0  0.000116  0.000000    197
2    45  0.000100  0.000044    196
3    90  0.000101  0.000000    178
4   135  0.000060  0.000051    180
5   180  0.000112  0.000000    186
6   225  0.000103  0.000000    185
7   270  0.000106  0.000051    188
8   315  0.000108  0.000044    191


In [259]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output # For use in Jupyter-like environments

# --- Helper Function Definition ---
def _process_dataset_for_Ic_By_plot(
    dataid, 
    angle, 
    angle_label, 
    fig, 
    IcMax_df, 
    ByOffset, 
    ByRange_filter, 
    f_factor, 
    data_loader_func
):
    """
    Processes data for a single dataset: loads data (or generates dummy data),
    filters it, aligns its 'param_value' start to 0 then applies ByOffset,
    calculates IcMax, updates IcMax_df, and adds a trace to the figure.

    Args:
        dataid (int): The ID of the dataset.
        angle (int or float): The angle in degrees.
        angle_label (str): A short label for the angle (e.g., "NE", "E").
        fig (go.Figure): The Plotly figure object to add traces to.
        IcMax_df (pd.DataFrame): DataFrame to store maximum Ic values, which will be modified.
        ByOffset (float): The offset to apply to the x-values AFTER aligning data start to 0.
        ByRange_filter (float): The range to filter 'param_value' (e.g., data between -ByRange_filter and +ByRange_filter).
        f_factor (float): The factor 'f' used in y-axis Ic_offset calculation (angle * 1e-6 * f_factor).
        data_loader_func (callable, optional): A function that takes 'dataid' and returns
                                              a pandas DataFrame with "param_value" and "Ic" columns.
                                              If it returns a tuple, it's assumed to be (fig1, fig2, fig3, Ic_df).
                                              If None, dummy data is generated.

    Returns:
        tuple: Updated (fig, IcMax_df)
    """
    print(f"Processing dataid: {dataid}, angle: {angle}° ({angle_label})")
    y_axis_Ic_offset = angle * 1e-6 * f_factor # Offset for y-values to separate traces visually
    
    Ic_df = None
    if data_loader_func is None:
        print(f"Warning: 'data_loader_func' not provided. Using dummy data for dataid {dataid}.")
        _data_points = 20 # Increased points for smoother dummy data
        # Generate base param values roughly centered around 0, with a span related to ByRange_filter
        _base_param_values = [(i - _data_points / 2 + 0.5) * (2 * ByRange_filter) / _data_points for i in range(_data_points)]
        # Add a small unique offset based on dataid to simulate different raw start points for testing alignment
        _param_values = [pv + (dataid * 1e-5) for pv in _base_param_values] # Reduced shift to keep within typical ByRange
        _ic_values = [abs(5e-6 - abs(pv - (dataid * 1e-5)) * 1e-3 + (angle / 90) * 1e-7) for pv in _param_values]
        Ic_df = pd.DataFrame({
            "param_value": _param_values,
            "Ic": _ic_values
        })
    else:
        returned_data = data_loader_func(dataid)
        if isinstance(returned_data, tuple) and len(returned_data) > 0 and isinstance(returned_data[-1], pd.DataFrame):
            # Assuming the last element of the tuple is the Ic_df
            Ic_df = returned_data[-1]
        elif isinstance(returned_data, pd.DataFrame):
            Ic_df = returned_data
        else:
            raise ValueError(
                f"data_loader_func for dataid {dataid} did not return data in an expected format "
                "(e.g., Ic_df DataFrame or a tuple ending with Ic_df)."
            )

    if Ic_df is None or "param_value" not in Ic_df.columns or "Ic" not in Ic_df.columns:
        print(f"Error: Ic_df for dataid {dataid} is invalid or missing required columns. Skipping.")
        return fig, IcMax_df

    # Filter the Ic_df based on the raw 'param_value' range.
    # This filter assumes 'param_value' is somewhat centered, or ByRange_filter defines absolute limits.
    Ic_df_filtered = Ic_df[
        (Ic_df["param_value"] < ByRange_filter) & (Ic_df["param_value"] > -ByRange_filter)
    ].copy()

    if Ic_df_filtered.empty:
        print(f"Warning: Ic_df_filtered is empty for dataid {dataid} after filtering with ByRange_filter={ByRange_filter}. "
              "Check filter, data source, or ByRange_filter value. Skipping this data point.")
        return fig, IcMax_df

    # --- Alignment Step ---
    min_param_value_in_filtered = Ic_df_filtered["param_value"].min()
    # Shift 'param_value's so the minimum (of the filtered set) is 0, then add the ByOffset.
    plot_x_values = (Ic_df_filtered["param_value"] - min_param_value_in_filtered) + ByOffset
    # --- End of Alignment Step ---

    if 'Ic' not in Ic_df_filtered.columns or not pd.api.types.is_numeric_dtype(Ic_df_filtered['Ic']):
        print(f"Error: 'Ic' column is missing or not numeric in Ic_df_filtered for dataid {dataid}.")
        return fig, IcMax_df
        
    # Find the original param_value (from filtered data) where Ic is maximal
    idx_max_ic = Ic_df_filtered["Ic"].idxmax()
    By_original_at_max_Ic = Ic_df_filtered.loc[idx_max_ic, "param_value"]
    Ic_max_value = Ic_df_filtered.loc[idx_max_ic, "Ic"]
    
    # Apply the same alignment transformation to this By_original for storing in IcMax_df
    By_aligned_for_plot = (By_original_at_max_Ic - min_param_value_in_filtered) + ByOffset
    
    new_row_data = [{"angle": angle, "IcMax": Ic_max_value, "By": By_aligned_for_plot, "dataid": dataid}]
    new_row_df = pd.DataFrame(new_row_data)
    IcMax_df = pd.concat([IcMax_df, new_row_df], ignore_index=True)
    
    fig.add_trace(go.Scatter(
        x=plot_x_values,
        y=Ic_df_filtered["Ic"] + y_axis_Ic_offset,
        mode='lines+markers',
        name=f'{angle}°{angle_label}',
        marker=dict(size=5),
        line=dict(width=2)
    ))
    
    # clear_output(wait=True) # Optional: uncomment if running in a context where clearing output per step is desired
    return fig, IcMax_df

# --- Main Plotting Function ---
def plot_aligned_Ic_vs_By(
    data_configurations,
    f_parameter,
    by_filter_range,
    actual_data_loader=None,
    plot_title="<b>Ic vs. By | In-Plane Field Angles (Aligned Start + Offset)</b>",
    xaxis_title_override=None, # Allow full override for xaxis title
    yaxis_title_override=None, # Allow full override for yaxis title
    legend_title="Angle",
    plot_width=1200,
    plot_height=800
):
    """
    Generates a Plotly figure showing Ic vs. By for multiple datasets.
    Each dataset's 'By' field (param_value) is aligned so its minimum starts at 0,
    then a specific ByOffset (from data_configurations) is applied.

    Args:
        data_configurations (list of dict): List of configurations for each dataset.
            Each dict must contain: "dataid", "angle", "label", "ByOffset".
        f_parameter (float): The 'f' factor used in y-axis Ic_offset calculation.
        by_filter_range (float): The range to filter 'param_value' before alignment 
                                 (e.g. data between -by_filter_range and +by_filter_range).
        actual_data_loader (callable, optional): Function to load real data. 
                                                 Takes dataid, returns Ic_df DataFrame or tuple ending in Ic_df.
                                                 If None, dummy data is used by the helper function.
        plot_title (str, optional): Title of the plot.
        xaxis_title_override (str, optional): Specific title for the X-axis. If None, a default is used.
        yaxis_title_override (str, optional): Specific title for the Y-axis. If None, a default is used.
        legend_title (str, optional): Title for the legend.
        plot_width (int, optional): Width of the plot in pixels.
        plot_height (int, optional): Height of the plot in pixels.

    Returns:
        tuple: (go.Figure, pd.DataFrame)
               - The Plotly figure object.
               - The DataFrame containing summarized IcMax values.
    """
    IcMax_df = pd.DataFrame(columns=["angle", "IcMax", "By", "dataid"])
    fig = go.Figure()

    print(f"Starting plot generation with f_parameter={f_parameter}, by_filter_range={by_filter_range}")

    for point_config in data_configurations:
        fig, IcMax_df = _process_dataset_for_Ic_By_plot(
            dataid=point_config["dataid"],
            angle=point_config["angle"],
            angle_label=point_config["label"],
            fig=fig,
            IcMax_df=IcMax_df,
            ByOffset=point_config["ByOffset"],
            ByRange_filter=by_filter_range,
            f_factor=f_parameter,
            data_loader_func=actual_data_loader
        )
    
    clear_output(wait=True) # Clear intermediate outputs from the loop
    print("All datasets processed. Finalizing plot.")

    if not IcMax_df.empty:
        fig.add_trace(go.Scatter(
            x=IcMax_df["By"],  # These 'By' values are already transformed
            y=IcMax_df["IcMax"] + IcMax_df["angle"] * 1e-6 * f_parameter, # Apply same y-offset logic
            mode='lines+markers',
            name='IcMax Trend',
            marker=dict(size=8, symbol='star', color='rgba(0,0,0,0.7)'),
            line=dict(width=2, dash='dash', color='rgba(0,0,0,0.7)')
        ))
    else:
        print("Warning: IcMax_df is empty. Skipping the IcMax trend plot.")

    # Determine axis titles
    xaxis_title = xaxis_title_override if xaxis_title_override is not None else "Out-of-Plane Field (By, aligned then offset) [T]"
    yaxis_title = yaxis_title_override if yaxis_title_override is not None else f"Critical Current (Ic) + Offset [A] (angle*{f_parameter:.2f}e-6)"


    fig.update_layout(
        title=plot_title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend_title=legend_title,
        hovermode="x unified",
        template="plotly_white",
        width=plot_width,
        height=plot_height,
    )
    
    print("Plot generation complete.")
    return fig, IcMax_df

# --- Example Usage ---
if __name__ == '__main__':
    # 1. Define your data configurations
    my_data_points = [
        {"dataid": 144, "angle": 0, "label": "O", "ByOffset": 0e-3},
        {"dataid": 197, "angle": 0, "label": "E", "ByOffset": 0.002}, # Simplified ByOffset for clarity
        {"dataid": 196, "angle": 45, "label": "NE", "ByOffset": 0.0025},
        {"dataid": 178, "angle": 90, "label": "N", "ByOffset": 0.0005},
        # Add more data points as needed
    ]

    # 2. Define parameters
    f_val = 0.25
    by_range = 0.02 # 10mT, ensure this is appropriate for your raw param_value scale

    # 3. (Optional) Define your actual data loader function
    # If you have a function like `plot_heatmaps` that returns (fig1, fig2, fig3, Ic_df)
    # or just Ic_df, you can pass it.
    # For this example, we'll use the dummy data by passing `None`.
    
    # Example of what your data loader might look like:
    # def my_actual_data_loader(dataid_to_load):
    #     print(f"LOADER: Loading real data for dataid {dataid_to_load}...")
    #     # ... your actual data loading logic ...
    #     # Example: fig1, fig2, fig3, loaded_ic_df = original_plot_heatmaps(dataid_to_load)
    #     # return fig1, fig2, fig3, loaded_ic_df 
    #     # OR if it just returns the DataFrame:
    #     # loaded_ic_df = load_ic_dataframe(dataid_to_load)
    #     # return loaded_ic_df
    #
    #     # For demonstration, let's make a dummy loader that returns a DataFrame
    #     _points = 30
    #     _raw_params = [(j - _points/2) * (2*by_range) / _points + (dataid_to_load/10000.0) for j in range(_points)]
    #     _raw_ics = [abs(6e-6 - abs(p)*0.8e-3) + (dataid_to_load/1000.0 * 1e-7) for p in _raw_params]
    #     dummy_df = pd.DataFrame({"param_value": _raw_params, "Ic": _raw_ics})
    #     return dummy_df # It's important it has "param_value" and "Ic"

    # For this run, we'll use the internal dummy data by setting actual_data_loader=None
    custom_data_loader = None # Replace with `my_actual_data_loader` if you have one

    # 4. Call the main plotting function
    print("Starting example script...")
    generated_fig, final_IcMax_df = plot_aligned_Ic_vs_By(
        data_configurations=my_data_points,
        f_parameter=f_val,
        by_filter_range=by_range,
        actual_data_loader=custom_data_loader, # Set to None to use default dummy data
        plot_title="<b>Demo: Aligned Ic vs. By Field Scan</b>"
    )

    # 5. Show the figure (if in an environment that supports it, like Jupyter)
    # In a script, you might save it to a file: generated_fig.write_html("aligned_plot.html")
    generated_fig.show()

    # 6. Print the resulting IcMax DataFrame
    print("\nFinal IcMax DataFrame (By values are aligned then offset):")
    print(final_IcMax_df.to_string())

All datasets processed. Finalizing plot.
Plot generation complete.



Final IcMax DataFrame (By values are aligned then offset):
  angle     IcMax      By dataid
0     0  0.000014  0.0000    144
1     0  0.000014  0.0020    197
2    45  0.000014  0.0025    196
3    90  0.000014  0.0005    178
